In [1]:
movie_likes

NameError: name 'movie_likes' is not defined

In [10]:
import pandas as pd
import numpy as np

def rec_movies(movie_id):
    
    movie_likes = pd.read_csv('./data/user_movie_likes.txt', header=-1)
    movie_likes.rename(columns={0: 'userid', 1: 'movieid'}, inplace=True)
    movies_df = pd.crosstab(values=movie_likes['movieid'], index=movie_likes['userid'], columns=movie_likes['movieid'], aggfunc='count').replace({np.nan: 0})
    user_movie = movies_df.values
    movie_user = user_movie.T
    movie_corr = np.matmul(movie_user, movie_user.T)
    movie_corr_df = pd.DataFrame(data=movie_corr, index=movies_df.columns, columns=movies_df.columns)
    
    return list(movie_corr_df[movie_id].sort_values(ascending=False)[1:4].index)

def rec_movie_user(user_id):
    
    movie_likes = pd.read_csv('./data/user_movie_likes.txt', header=-1)
    movie_likes.rename(columns={0: 'userid', 1: 'movieid'}, inplace=True)
    movies_df = pd.crosstab(values=movie_likes['movieid'], index=movie_likes['userid'], columns=movie_likes['movieid'], aggfunc='count').replace({np.nan: 0})
    user_movie = movies_df.values
    user_corr = np.matmul(user_movie, user_movie.T)
    user_corr_df = pd.DataFrame(data=user_corr, index=movies_df.index, columns=movies_df.index)
    
    sim_users = list(user_corr_df[101].sort_values(ascending=False).index[1:4])
    users_df = movies_df.transpose()
    
    movies_unseen = users_df[sim_users].apply(sum, axis=1) - 3*users_df[user_id]
    
    return list(movies_unseen.sort_values(ascending=False).index[:3])



In [18]:
from collections import Counter
mu = movie_likes.groupby('movieid')['userid'].apply(lambda i: list(i))
um = movie_likes.groupby('userid')['movieid'].apply(lambda i: list(i))

def sim_mov(movieid):
    l = []
    for userid in mu[movieid]:
        l.extend(um[userid])
    l = [i for i in l if i != movieid]
    return [i[0] for i in Counter(l).most_common(3)]

def rec_mov(userid):
    users = []
    for movieid in um[userid]:
        users.extend(mu[movieid])
    l = []
    for userid in set(users):
        l.extend(um[userid])
    l = [i for i in l if i not in um[userid]]
    return [i[0] for i in Counter(l).most_common(3)]